## 2021: Week 2 

Building on from Week 1's challenge, we are going to take your data prep skills on one step further. The next steps we are introducing this week are:

1. Aggregation - changing the level of granularity of your data. The combination of the categorical fields often sets what each row represents so aggregating data changes this. In Tableau Prep this is different from how we aggregate in Tableau Desktop.

2. Calculations - If the value or variable that you need to use isn't in your data set, you will often be able to create it from the other data fields you do have.

### Challenge

This week we are looking at the different Brands of bikes available in our stores. We need to understand what are the most popular sellers and do the customers of the different brands have the same experience to other customers. 

We are creating simple summaries this week to get a quick, tabular view of the answers. If you want to visualise the data to highlight those trends even more clearly, go for it! 

### Input

![img](https://1.bp.blogspot.com/-z_heCV23gPQ/X_rk1Pa7a8I/AAAAAAAACGM/v4DDRd218DcDw71jZPbyVVGgxkee-wSsQCLcBGAsYHQ/w640-h214/Screenshot%2B2021-01-10%2Bat%2B11.27.51.png)

### Requirements

- Input the data 
- Clean up the Model field to leave only the letters to represent the Brand of the bike 
- Workout the Order Value using Value per Bike and Quantity.
- Aggregate Value per Bike, Order Value and Quantity by Brand and Bike Type to form: 
    - Quantity Sold
    - Order Value
    - Average Value Sold per Brand, Type
- Calculate Days to ship by measuring the difference between when an order was placed and when it was shipped as 'Days to Ship'
- Aggregate Order Value, Quantity and Days to Ship by Brand and Store to form:
    - Total Quantity Sold
    - Total Order Value
    - Average Days to Ship
- Round any averaged values to one decimal place to make the values easier to read
- Output both data sets

### Output

#### Sales by Brand and Type

![img](https://1.bp.blogspot.com/-pIoOV-rm3JM/X_8FXjz9FbI/AAAAAAAACGs/mWo2N0mn1vwHqnH-_n8X60M_kvKiIUCwACLcBGAsYHQ/w640-h244/Screenshot%2B2021-01-13%2Bat%2B14.35.41.png)

5 Data Fields:
- Brand
- Bike Type
- Quantity Sold
- Order Value
- Avg Bike Value per Brand

15 Rows (16 including headers)

#### Sales by Brand and Store

![img2](https://1.bp.blogspot.com/-vKiMGxugMJg/X_8FpFFpm5I/AAAAAAAACG0/BVU8dfchDR8z_3Oyfc8ruJ7j9S1OSVgqQCLcBGAsYHQ/w640-h264/Screenshot%2B2021-01-13%2Bat%2B14.37.14.png)

5 Data Fields
- Brand
- Store
- Total Quantity Sold
- Total Order Value
- Avg Days to Ship

25 Rows (26 including headers)

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [169]:
# Input the data
df = pd.read_csv("./data/PD 2021 Wk 2 Input - Bike Model Sales.csv")
df.head()

,Bike Type,Store,Order Date,Quantity,Value per Bike,Shipping Date,Model
0,Mountain,Manchester,15/05/2020,4,1543,01/06/2020,GIA31292/003
1,Gravel,Manchester,16/06/2020,2,2076,24/06/2020,GIA21312/001
2,Road,Birmingham,04/05/2020,1,2616,13/05/2020,GIA94221/129
3,Gravel,York,05/09/2020,2,1359,19/09/2020,GIA12442/120
4,Gravel,Birmingham,28/03/2020,4,1599,04/04/2020,GIA12492/123


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Bike Type       2000 non-null   object
 1   Store           2000 non-null   object
 2   Order Date      2000 non-null   object
 3   Quantity        2000 non-null   int64 
 4   Value per Bike  2000 non-null   int64 
 5   Shipping Date   2000 non-null   object
 6   Model           2000 non-null   object
dtypes: int64(2), object(5)
memory usage: 109.5+ KB


In [171]:
df.describe(include="number").T

,count,mean,std,min,25%,50%,75%,max
Quantity,2000.0,2.9975,1.430565,1.0,2.0,3.0,4.0,5.0
Value per Bike,2000.0,2380.3440,939.008323,752.0,1581.5,2337.5,3193.0,4000.0


In [172]:
df.describe(include="object").T

,count,unique,top,freq
Bike Type,2000,3,Mountain,690
Store,2000,5,Birmingham,435
Order Date,2000,364,15/02/2020,16
Shipping Date,2000,373,10/06/2020,13
Model,2000,421,83481/012KONA,36


In [173]:
# Clean up the Model field to leave only the letters to represent the Brand of the bike
df["Model"].value_counts()

83481/012KONA    36
93481/014KONA    36
12123/014KONA    36
84123/192KONA    36
02381/123KONA    36
                 ..
942SPEC34360      3
120SPEC13510      3
220SPEC12421      3
012SPEC24040      3
943SPEC24926      3
Name: Model, Length: 421, dtype: int64

In [174]:
df["Brand"] = df["Model"].str.extract(r'([a-zA-Z]+)')
df["Brand"].value_counts()

SPEC    963
GIA     392
KONA    312
BROM    228
ORRO    105
Name: Brand, dtype: int64

In [175]:
# Workout the Order Value using Value per Bike and Quantity.
df["Order Value"] = df["Value per Bike"] * df["Quantity"]
df

,Bike Type,Store,Order Date,Quantity,Value per Bike,Shipping Date,Model,Brand,Order Value
0,Mountain,Manchester,15/05/2020,4,1543,01/06/2020,GIA31292/003,GIA,6172
1,Gravel,Manchester,16/06/2020,2,2076,24/06/2020,GIA21312/001,GIA,4152
2,Road,Birmingham,04/05/2020,1,2616,13/05/2020,GIA94221/129,GIA,2616
3,Gravel,York,05/09/2020,2,1359,19/09/2020,GIA12442/120,GIA,2718
4,Gravel,Birmingham,28/03/2020,4,1599,04/04/2020,GIA12492/123,GIA,6396
...,...,...,...,...,...,...,...,...,...
1995,Road,Manchester,02/06/2020,2,3504,20/06/2020,102SPEC84233,SPEC,7008
1996,Mountain,York,07/09/2020,2,1109,21/09/2020,012SPEC93591,SPEC,2218
1997,Road,London,10/12/2020,3,1032,11/12/2020,943SPEC24922,SPEC,3096
1998,Gravel,London,10/02/2020,2,2303,11/02/2020,429SPEC21322,SPEC,4606


In [176]:
# Aggregate Value per Bike, Order Value and Quantity by Brand and Bike Type to form
sales_by_brand_type = (df.
     groupby(["Brand", "Bike Type"])
     [["Quantity", "Order Value"]]
     .agg("sum")
     .assign(Avg_Bike_value_Sold_per_Brand_type=lambda df_: (df_["Order Value"] / df_["Quantity"]).round(2))
     .reset_index()
     .rename(columns={"Avg_Bike_value_Sold_per_Brand_type": "Avg Bike Value per Brand",
                      "Quantity": "Quantity Sold"})
)
sales_by_brand_type

,Brand,Bike Type,Quantity Sold,Order Value,Avg Bike Value per Brand
0,BROM,Gravel,186,433885,2332.72
1,BROM,Mountain,277,674770,2435.99
2,BROM,Road,257,656539,2554.63
3,GIA,Gravel,323,733087,2269.62
4,GIA,Mountain,425,1021329,2403.13
5,GIA,Road,407,896695,2203.18
6,KONA,Gravel,324,791841,2443.95
7,KONA,Mountain,330,820537,2486.48
8,KONA,Road,273,647684,2372.47
9,ORRO,Gravel,151,411644,2726.12


In [177]:
import plotly.express as px

fig = px.bar(sales_by_brand_type, x="Brand", y="Order Value", color="Bike Type", barmode="group",
             title="Sales Values per Brand")
fig.show()

In [178]:
fig = px.bar(sales_by_brand_type, x="Brand", y="Quantity Sold", title="Quantity sold per Type",
             color="Bike Type")
fig.show()

In [179]:
# Calculate Days to ship by measuring the difference between when an order was placed and when it was shipped as 'Days to Ship'
df["Order Date"] = pd.to_datetime(df["Order Date"], format="%d/%m/%Y")
df["Shipping Date"] = pd.to_datetime(df["Shipping Date"], format="%d/%m/%Y")
df["Days to Ship"] = df["Shipping Date"] - df["Order Date"]
df.head()

,Bike Type,Store,Order Date,Quantity,Value per Bike,Shipping Date,Model,Brand,Order Value,Days to Ship
0,Mountain,Manchester,2020-05-15,4,1543,2020-06-01,GIA31292/003,GIA,6172,17 days
1,Gravel,Manchester,2020-06-16,2,2076,2020-06-24,GIA21312/001,GIA,4152,8 days
2,Road,Birmingham,2020-05-04,1,2616,2020-05-13,GIA94221/129,GIA,2616,9 days
3,Gravel,York,2020-09-05,2,1359,2020-09-19,GIA12442/120,GIA,2718,14 days
4,Gravel,Birmingham,2020-03-28,4,1599,2020-04-04,GIA12492/123,GIA,6396,7 days


In [180]:
# Aggregate Order Value, Quantity and Days to Ship by Brand and Store to form
sales_brand_store= (df
    .groupby(["Brand", "Store"]).agg(Total_Order_Value=pd.NamedAgg(column="Order Value", aggfunc="sum"),
                                     Total_Quantity_Sold=pd.NamedAgg(column="Quantity", aggfunc="sum"),
                                     Avg_Days_to_Ship=pd.NamedAgg(column="Days to Ship", aggfunc="mean"))
    .reset_index()
    .assign(Avg_Days_to_Ship=lambda df_: df_.Avg_Days_to_Ship.dt.ceil("D"))
    .rename(columns=lambda name_: name_.replace("_", " "))
)
sales_brand_store

,Brand,Store,Total Order Value,Total Quantity Sold,Avg Days to Ship
0,BROM,Birmingham,349759,155,12 days
1,BROM,Leeds,389116,150,10 days
2,BROM,London,324635,133,11 days
3,BROM,Manchester,339832,137,11 days
4,BROM,York,361852,145,10 days
5,GIA,Birmingham,581733,269,10 days
6,GIA,Leeds,460151,203,12 days
7,GIA,London,548821,228,11 days
8,GIA,Manchester,466613,204,12 days
9,GIA,York,593793,251,11 days


In [181]:
sales_by_brand_type.to_csv("./output/Week2_output_1.csv")
sales_brand_store.to_csv("./output/Week2_output_2.csv")